### 🚀 Web Data Extraction with WaterCrawl, LiteLLM, and Rank-BM25

Welcome to this **step-by-step Jupyter Notebook** where we explore goal-oriented web crawling. This tutorial will guide you through using WaterCrawl to map a website, filter URLs with Rank-BM25 and LLMs, scrape content, and analyze it to meet a specific objective.

#### What’s inside?
| 🔧 Component | 💡 Why we’re using it |
|--------------|----------------------|
| **WaterCrawl** | For mapping websites and scraping content with precision. |
| **LiteLLM** | To interact with various LLM providers for strategy generation and content analysis. |
| **Rank-BM25** | For efficient keyword-based URL filtering and ranking. |

#### Notebook Flow 🗺️
1. **Setup**: Install dependencies and configure API keys.
2. **Initialization**: Set up the target URL and objective.
3. **Sitemap Extraction**: Use WaterCrawl to fetch the website's sitemap.
4. **URL Filtering**: Apply Rank-BM25 and LLM-based filtering to select relevant URLs.
5. **Content Scraping**: Scrape content from top URLs using WaterCrawl.
6. **Content Analysis**: Analyze scraped content with LLMs to meet the objective.
7. **Results**: Compile and display the final structured response.

#### Why you’ll ❤️ this approach
- **Efficiency**: Quickly process complex websites.
- **Flexibility**: Switch between LLM providers for different tasks.
- **Precision**: Combine keyword and semantic analysis for accurate results.

> **Tip:** If you’re new to WaterCrawl, check out the [WaterCrawl Documentation](https://docs.watercrawl.dev/intro) for more details.

Ready? Let’s start by setting up our environment! 🏁

##### ➡️ **Install all the dependencies:**

In [ ]:
!pip install -r requirements.txt

##### ➡️ **API keys you’ll need (grab these first!)** 

| Service | What it’s for | Where to generate |
|---------|---------------|-------------------|
| **WaterCrawl** | Auth for crawling endpoints | <https://app.watercrawl.dev/dashboard/api-keys> |
| **OpenAI/Other LLMs** | LLM interactions | Depends on provider (e.g., OpenAI, Anthropic) |

---
**Option 1 – Keep it clean: Use a `.env` file** ⚠️

Create the file **once**, store your keys, and everything else “just works”.

```python
# Create .env file
env_text = """
WATERCRAWL_API_KEY=your_watercrawl_api_key_here
OPENAI_API_KEY=your_openai_api_key_here
ANTHROPIC_API_KEY=your_anthropic_api_key_here
DEEPSEEK_API_KEY=your_deepseek_api_key_here
""".strip()

with open(".env", "w") as f:
    f.write(env_text)
print(".env file created — now edit it with your real keys ✏️")
```

**Option 2 – Quick-and-dirty: Hard-code in the notebook** ⚠️

Not recommended — anyone who sees or commits the notebook can read your keys.

WATERCRAWL_API_KEY=your_watercrawl_api_key_here
OPENAI_API_KEY=your_openai_api_key_here
ANTHROPIC_API_KEY=your_anthropic_api_key_here
DEEPSEEK_API_KEY=your_deepseek_api_key_here

##### ➡️ **If you’re using a `.env` file, load the API keys with dotenv**

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()  # pulls everything from .env for any API keys for LLMS


WATERCRAWL_API_KEY = os.environ.get("WATERCRAWL_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.environ.get("ANTHROPIC_API_KEY")
DEEPSEEK_API_KEY = os.environ.get("DEEPSEEK_API_KEY")


##### ➡️ **Import necessary packages:**

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('./objective_crawler'))
from core import ObjectiveCrawler
from clients import WaterCrawler, LLMClient
from config import DEFAULT_MODEL, DEFAULT_TOP_K, DEFAULT_STRATEGIES

#### ➡️ **Set up the Objective and URL:**
Define the website URL you want to crawl and the objective you want to achieve.

In [ ]:
# Example URL and Objective
target_url = "https://watercrawl.dev/"
objective = "Find pricing information"

# You can modify these to test with your own inputs
print(f"Target URL: {target_url}")
print(f"Objective: {objective}")

#### ➡️ **Initialize the Crawler:**
Set up the crawler with configurable options for LLM model, number of URLs to scrape, and search strategies.

In [ ]:
# Initialize LLM Client with your preferred model
llm_client = LLMClient(model=DEFAULT_MODEL, api_key=LITELLM_API_KEY)

# Initialize WaterCrawler with your API key
water_crawler = WaterCrawler(api_key=WATERCRAWL_API_KEY)

# Create ObjectiveCrawler instance
crawler = ObjectiveCrawler(
    water_crawler=water_crawler,
    llm_client=llm_client,
    top_k=DEFAULT_TOP_K,
    num_strategies=DEFAULT_STRATEGIES
)

print("Crawler initialized with model:", DEFAULT_MODEL)
print(f"Top K URLs to scrape: {DEFAULT_TOP_K}")
print(f"Number of search strategies: {DEFAULT_STRATEGIES}")

#### ➡️ **Fetch Sitemap with WaterCrawl:**
Use WaterCrawl to map the entire website and retrieve all URLs.

In [ ]:
# Fetch sitemap
sitemap_urls = crawler.get_sitemap(target_url)

print(f"Total URLs in sitemap: {len(sitemap_urls)}")
print("Sample URLs:", sitemap_urls[:5] if sitemap_urls else [])

#### ➡️ **Generate Search Strategies with LLM:**
Generate multiple search strategies to filter URLs based on the objective.

In [ ]:
# Generate search strategies
strategies = crawler.generate_search_strategies(objective)

print("Generated Search Strategies:")
for i, strategy in enumerate(strategies, 1):
    print(f"{i}. {strategy}")

#### ➡️ **Filter URLs with Rank-BM25:**
Use Rank-BM25 to perform keyword-based filtering and ranking of URLs.

In [ ]:
# Filter URLs with BM25
filtered_urls_bm25 = crawler.filter_urls_bm25(sitemap_urls, objective, strategies)

print(f"URLs after BM25 filtering: {len(filtered_urls_bm25)}")
print("Top URLs:", filtered_urls_bm25[:5] if filtered_urls_bm25 else [])

#### ➡️ **Further Filter URLs with LLM:**
Use an LLM to refine the list of URLs based on relevance to the objective.

In [ ]:
# Filter URLs with LLM
filtered_urls_llm = crawler.filter_urls_llm(filtered_urls_bm25, objective)

print(f"URLs after LLM filtering: {len(filtered_urls_llm)}")
print("Top URLs for scraping:", filtered_urls_llm)

#### ➡️ **Scrape Content from Top URLs:**
Scrape the content from the selected URLs using WaterCrawl.

In [ ]:
# Scrape content
scraped_contents = crawler.scrape_urls(filtered_urls_llm)

print(f"Scraped content from {len(scraped_contents)} URLs")
for url, content in scraped_contents.items():
    print(f"URL: {url}")
    print(f"Content length: {len(content) if content else 0} characters")

#### ➡️ **Analyze Content with LLM:**
Analyze the scraped content to extract information relevant to the objective.

In [ ]:
# Analyze content
individual_analyses = crawler.analyze_content(scraped_contents, objective)

print("Individual Analyses:")
for url, analysis in individual_analyses.items():
    print(f"URL: {url}")
    print(f"Analysis: {analysis[:200]}...")

#### ➡️ **Compile Final Results:**
Compile all analyses into a structured JSON response that answers the objective.

In [ ]:
# Compile final result
final_result = crawler.generate_final_result(objective, individual_analyses)

print("Final Result:")
import json
print(json.dumps(final_result, indent=2))

### 🌟 Conclusion
Congratulations! You've successfully used WaterCrawl, LiteLLM, and Rank-BM25 to crawl a website, filter URLs, scrape content, and analyze it to meet a specific objective. 

#### What you’ve learned:
- How to set up and configure tools for web data extraction.
- Mapping a website and filtering URLs with BM25 and LLMs.
- Scraping and analyzing content to answer targeted questions.

#### Next Steps:
- Experiment with different URLs and objectives.
- Try different LLM models for strategy generation and analysis.
- Scale up by integrating with other tools or larger datasets.

If you found this tutorial helpful, consider starring the [WaterCrawl repo](https://github.com/watercrawl/watercrawl) on GitHub! ⭐